In [ ]:
!pip install easyocr 
!pip install opencv-python
!pip install numpy
!pip install pandas
!pip install matplotlib

In [1]:
import cv2
import numpy as np
import pandas as pd
from collections import deque
from collections import Counter
import time
import os
import matplotlib.pyplot as plt
from PreProcessing import License_Detection

In [2]:
class TrackedObject:
    def __init__(self, id, bbox):
        self.id = id
        self.bbox = bbox
        self.centroid_history = deque(maxlen=5)
        self.speed_history = deque(maxlen=5)
        self.last_seen = time.time()

    def update(self, bbox, speed):
        self.bbox = bbox
        self.speed_history.append(speed)
        centroid = ((bbox[0] + bbox[2]) // 2, (bbox[1] + bbox[3]) // 2)
        self.centroid_history.append(centroid)
        self.last_seen = time.time()

    def get_average_speed(self):
        if len(self.speed_history) > 0:
            return np.mean(self.speed_history)
        else:
            return 0

def is_valid_contour(contour, min_area=5000, min_aspect_ratio=0.5, max_aspect_ratio=3.0):
    if cv2.contourArea(contour) < min_area:
        return False

    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h

    if aspect_ratio < min_aspect_ratio or aspect_ratio > max_aspect_ratio:
        return False

    return True

def estimate_speeds(video_path):
    prev_gray = None
    frame_count = 0
    fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video!")
        return
    



    fps = cap.get(cv2.CAP_PROP_FPS)
    time_interval = 1 / fps

    tracked_objects = {}
    next_object_id = 0
    disappear_time_limit = 0.15

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilate_iterations = 15 #12
    erode_iterations = 2  #4
    

    
    id_counter = Counter() 
    ids_over_threshold = Counter()   
    pre_ids_over_threshold= Counter()
    Save_ID = []  
    Save_License = {}
    threshold_major_id=30
    Speed_result = 0
    Speed_results = []
    Distance=20
    config=0.75



    output_image_dir = "OutputImage"
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)

    def save_detected_vehicle(frame, obj,id_counter):
        x, y, x2, y2 = obj.bbox
        vehicle_image = frame[y:y2, x:x2]
        filename = f"vehicle_{obj.id}_{id_counter}.png"  
        cv2.imwrite(os.path.join(output_image_dir, filename), vehicle_image)


    fps=30
    output_video_path = "OutputVideo/Detected-Vehicles.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (1920, 1080))


    while True:
        ret, frame = cap.read()
        if not ret:
            print("No frames captured or end of video reached!")
            break
        


        frame = cv2.resize(frame, (1920, 1080))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fgmask = fgbg.apply(gray)

        _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
        fgmask = cv2.dilate(fgmask, kernel, iterations=dilate_iterations)
        fgmask = cv2.erode(fgmask, kernel, iterations=erode_iterations)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=1) 

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                if not is_valid_contour(contour):
                    continue

                x, y, w, h = cv2.boundingRect(contour)
                avg_flow_x = np.mean(flow[y:y+h, x:x+w, 0])

                conversion_factor = 0.05
                car_speed = abs(avg_flow_x) * conversion_factor / time_interval
                car_speed = car_speed * 3.6

                centroid = (x + w // 2, y + h // 2)

                matched = False
                for obj in tracked_objects.values():
                    if np.linalg.norm(np.array(obj.centroid_history[-1]) - np.array(centroid)) < 50:
                        obj.update((x, y, x+w, y+h), car_speed)
                        matched = True
                        break

                if not matched:
                    new_object = TrackedObject(next_object_id, (x, y, x+w, y+h))
                    new_object.update((x, y, x+w, y+h), car_speed)
                    tracked_objects[next_object_id] = new_object
                    next_object_id += 1

            current_time = time.time()
            for obj_id in list(tracked_objects.keys()):
                if current_time - tracked_objects[obj_id].last_seen > disappear_time_limit:
                    del tracked_objects[obj_id]


            
            for obj in tracked_objects.values():

                if len(obj.centroid_history) > 0:
                    avg_speed = obj.get_average_speed()
                    x, y, x2, y2 = obj.bbox
                    cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID: {obj.id} Speed: {avg_speed:.2f} km/h", (x, y - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

          
            for obj in tracked_objects.values():
                
            
                if id_counter[obj.id] > threshold_major_id:

                    pre_ids_over_threshold[obj_id] = ids_over_threshold[obj_id]
                    save_detected_vehicle(frame, obj, ((id_counter[obj.id])-threshold_major_id)) 
                    ids_over_threshold[obj.id] +=1 
                    
                    if len(Save_ID)>=2:
                      last_item_ID = Save_ID[-1]
                      second_last_ID = Save_ID[-2]
                      if last_item_ID != obj.id  and second_last_ID != obj.id:
                          Save_ID.append(obj.id)
                    elif len(Save_ID)==1:
                        last_item_ID = Save_ID[-1]
                        if last_item_ID != obj.id:
                          Save_ID.append(obj.id)
                    else:
                         Save_ID.append(obj.id)  


                id_counter[obj.id] += 1 


        
            for obj_id in Save_ID:
                 
                if ids_over_threshold[obj_id] > pre_ids_over_threshold[obj_id]:
                    pre_ids_over_threshold[obj_id] = ids_over_threshold[obj_id]

                else:
                
                    Speed_result = (Distance/((ids_over_threshold[obj_id]+threshold_major_id+1) / 30))*3.6
                    
                    if Speed_result not in Speed_results:
                            Speed_results.append(Speed_result)

                    min_length = min(len(Save_ID), len(Speed_results))
                    
                    for i in range(min_length):
                        
                        if Speed_results[i] > 30:
                           
                           Color=(0, 0, 255)
                        else:

                           Color=(0, 255, 0)

                        cv2.putText(frame, f"Max Allowed Speed 30 km/h", (0,60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
                        cv2.putText(frame, f"ID: {Save_ID[i]} Speed: {Speed_results[i]:.2f} km/h", (0, (150+ ((2*i)-1)*30)), 
                                                                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, Color, 2, cv2.LINE_AA)
                        
                        if  Save_ID[i] in Save_License:
                            cv2.putText(frame, f"ID: {Save_ID[i]} License: {Save_License[Save_ID[i]]}", (0, (150 + (2*i * 30))), 
                                                                          cv2.FONT_HERSHEY_SIMPLEX, 0.8, Color, 2, cv2.LINE_AA)

                        else:    
                            Save_License[Save_ID[i]] = License_Detection(Save_ID[i], int(ids_over_threshold[Save_ID[i]]*config))
                            print(f"ID: {Save_ID[i]} Registering...")
                            
        prev_gray = gray
        output_video.write(frame)
        
                                            
    print("Object IDs and their counts:")
    for id, count in ids_over_threshold.items():  
            print(f"ID: {id}, Count: {count}")


    print(">>>>>>> Speed")
    for id  in Speed_results:  
            print(id)           


    print(">>>>>>> ID")
    for id  in Save_ID:  
            print(id)

    
    output_video.release()
    cap.release()
  

video_path = "inputVideo.mp4"
estimate_speeds(video_path)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


R6O6L
ID: 1 Registering...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


AE67OS
ID: 2 Registering...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


...

ID: 24 Registering...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


A968B6
ID: 25 Registering...


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


AV6190
ID: 28 Registering...
No frames captured or end of video reached!
Object IDs and their counts:
ID: 1, Count: 138
ID: 2, Count: 97
ID: 10, Count: 34
ID: 24, Count: 83
ID: 25, Count: 82
ID: 28, Count: 81
>>>>>>> Speed
12.781065088757396
16.875
33.23076923076923
18.947368421052634
19.115044247787612
19.285714285714285
>>>>>>> ID
1
2
10
24
25
28
